#Mengimpor Library

In [1]:
# Library untuk Model
import numpy as np
import tensorflow as tf
import random

# Kontekstualisasi dan pekerjaan NLP
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# Library pendukung
import json
import pickle
import warnings
import curses

warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
# Mengecek versi Tensorflow
print(tf. __version__)

2.12.0


#Pengolahan Dataset

In [3]:
print("Memproses dataset intents.....")
with open('./dataset/intents.json') as json_data:
    intents = json.load(json_data)

Memproses dataset intents.....


In [4]:
words = []
classes = []
documents = []
ignore_words = ['?']
print("Mengiterasi isi dataset untuk mengubahnya menjadi words, class, documents, dan ignore_words.......")
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # Tokenisasi tiap kata dalam kalimat
        w = nltk.word_tokenize(pattern)
        # Menambah kata ke dalam array words
        words.extend(w)
        # Menambah dokumen ke dalam korpus (pelatihan/uji)
        documents.append((w, intent['tag']))
        # Menambah tag ke dalam array class
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

Mengiterasi isi dataset untuk mengubahnya menjadi words, class, documents, dan ignore_words.......


In [5]:
print("Meng-stemming, menggunakan huruf kecil dan menghapus duplikat kata.......")
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

Meng-stemming, menggunakan huruf kecil dan menghapus duplikat kata.......
291 documents
33 classes ['ac_arti', 'ac_bocor', 'ac_freon', 'ac_kompresor', 'ac_kotor', 'ac_merek', 'ac_rusak', 'complaint', 'create_account', 'hp_baterai', 'hp_bluetooth', 'hp_bootlop', 'hp_iccharger', 'hp_lcd', 'hp_overheat', 'hp_wifi', 'laptop_audio_problems', 'laptop_battery_issues', 'laptop_blue_screen', 'laptop_charging_issues', 'laptop_driver_errors', 'laptop_hard_drive_failure', 'laptop_keyboard_not_working', 'laptop_overheating', 'laptop_screen_flickering', 'laptop_screen_problems', 'laptop_slow_performance', 'laptop_startup_issues', 'laptop_usb_port_issues', 'laptop_virus_infection', 'laptop_wifi_issues', 'penutup', 'salam']
296 unique stemmed words [',', 'ac', 'ad', 'adalah', 'adapt', 'admin', 'air', 'ak', 'aku', 'akun', 'ap', 'aplikas', 'artiny', 'assalamualaik', 'ata', 'bag', 'bagaiman', 'bantuanny', 'banyak', 'baru', 'bat', 'batera', 'baterainy', 'bau', 'beberap', 'bengkak', 'berair', 'berfungs', '

In [6]:
print("Membuat data untuk model.....")
training = []
output = []
print("Membuat list kosong untuk output.....")
output_empty = [0] * len(classes)

print("Membuat training set dan Bag of Words ....")
for doc in documents:
    # Inisialisasi Bag of Words
    bag = []
    # Daftar kata-kata yang ditokenisasi untuk pattern
    pattern_words = doc[0]
    # Stem setiap kata
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # Membuat array Bag of Words
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # Outputnya '0' untuk setiap tag dan '1' untuk tag saat ini
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

Membuat data untuk model.....
Membuat list kosong untuk output.....
Membuat training set dan Bag of Words ....


#Membuat Training dan Test Set

In [7]:
print("Mengacak dan mengubah menjadi numpy array untuk pemrosesan yang lebih cepat......")
random.shuffle(training)
training = np.array(training)

print("Membuat list train dan test.....")
train_x = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

print("Shape train_x:", np.shape(train_x))
print("Shape train_y:", np.shape(train_y))

Mengacak dan mengubah menjadi numpy array untuk pemrosesan yang lebih cepat......
Membuat list train dan test.....
Shape train_x: (291, 296)
Shape train_y: (291, 33)


#Membuat Model Sequential

In [8]:
from tensorflow.keras.initializers import TruncatedNormal

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(8, input_shape=(len(train_x[0]),), activation='relu', kernel_initializer=TruncatedNormal()))
model.add(tf.keras.layers.Dense(len(train_y[0]), activation='softmax', kernel_initializer=TruncatedNormal()))

# Kompilasi dan summary model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 2376      
                                                                 
 dense_1 (Dense)             (None, 33)                297       
                                                                 
Total params: 2,673
Trainable params: 2,673
Non-trainable params: 0
_________________________________________________________________


In [9]:
print("Melakukan training model.......")
# Ensure the input shape matches the number of input features

model.fit(train_x, train_y, batch_size=8, epochs=200)

Melakukan training model.......
Epoch 1/200
37/37 [==============================] - 1s 2ms/step - loss: 3.4945 - acc: 0.0275
Epoch 2/200
37/37 [==============================] - 0s 2ms/step - loss: 3.4838 - acc: 0.0962
Epoch 3/200
37/37 [==============================] - 0s 2ms/step - loss: 3.4668 - acc: 0.1271
Epoch 4/200
37/37 [==============================] - 0s 2ms/step - loss: 3.4375 - acc: 0.1787
Epoch 5/200
37/37 [==============================] - 0s 2ms/step - loss: 3.3927 - acc: 0.1924
Epoch 6/200
37/37 [==============================] - 0s 2ms/step - loss: 3.3322 - acc: 0.2131
Epoch 7/200
37/37 [==============================] - 0s 2ms/step - loss: 3.2577 - acc: 0.2268
Epoch 8/200
37/37 [==============================] - 0s 2ms/step - loss: 3.1743 - acc: 0.2234
Epoch 9/200
37/37 [==============================] - 0s 2ms/step - loss: 3.0859 - acc: 0.2371
Epoch 10/200
37/37 [==============================] - 0s 2ms/step - loss: 2.9947 - acc: 0.2440
Epoch 11/200
37/37 [=======

In [10]:
# Menyimpan model dan pickle
print("Melakukan save model.......")
model.save('model.h5')

print("Melakukan save pickle..........")
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

Melakukan save model.......
Melakukan save pickle..........


#Preprocessing input data dari user

In [11]:
# Melakukan loading pickle
print("Melakukan load pickle.....")
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']


with open('dataset/intents.json') as json_data:
    intents = json.load(json_data)

print("Melakukan load model......")
# Melakukan loading model
model = tf.keras.models.load_model('./model.h5')

Melakukan load pickle.....
Melakukan load model......


In [12]:
def clean_up_sentence(sentence):
    # Melakukan tokenisasi dan stemming
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# Me-return array Bag of Words: True atau False dan 0 atau 1 untuk setiap kata di Bag of Words yang ada di kalimat user
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return np.array(bag).reshape(1, -1)  # Reshape to (1, num_features)

#Probabilitas maksimum untuk dihapus dari hasil
ERROR_THRESHOLD = 0.25

def classify(sentence):
    # Melakukan prediksi menggunakan model
    results = model.predict([bow(sentence, words)])[0]
    # Menghapus hasil yang dibawah treshhold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # Mengsortir model berdasarkan probabilitas tertinggi
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1])) #Tuppl -> Intent and Probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # Menyesuaikan tag yang cocok
    if results:
        while results:
            for i in intents['intents']:
                if i['tag'] == results[0][0]:
                    # Mengacak response
                    return print(random.choice(i['responses']))

            results.pop(0)

#Pengujian Model

In [13]:
while True:
    input_data = input("You- ")
    if input_data.lower() in ['exit', 'selamat tinggal']:
        break
    else:
        answer = response(input_data)

You- Hai
1/1 [==============================] - 0s 80ms/step
Halo, Selamat datang di JasaIn Chatbot!. Jika kamu mempunyai kendala, silahkan deskripsikan kendalamu ya!
You- Bye
1/1 [==============================] - 0s 24ms/step
Sama-sama, Senang Bisa membantu!, Terimakasih telah menghubungi kami
You- AC rusak
1/1 [==============================] - 0s 20ms/step
Kerusakan pada AC dapat disebabkan oleh hal antara lain, part yang tidak dibersihkan, freon habis, bocor, kompresor rusak.
You- Laptop layar
1/1 [==============================] - 0s 21ms/step
Jika layar laptop tidak menyala atau tetap hitam, periksa kabel konektor layar dan pastikan tidak ada masalah dengan inverter backlight. Jika masih tidak berhasil, ada kemungkinan kerusakan pada panel layar dan perlu diperbaiki oleh teknisi.
You- Merek AC
1/1 [==============================] - 0s 36ms/step
Menurut Tokopedia, merek AC yang terbaik antara lain adalah Daikin, Sharp, Panasonic, Gree, LG, Samsung, AUX, Changhong, Mitshubishi, De

#Konversi TFLite

In [16]:
# Konversi .h5 model menjadi TFLite model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Melakukan save TFLite model
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)